In [1]:
!cp '/content/drive/MyDrive/Colab Notebooks/corpus_mecab.txt.zip' ./

In [2]:
!unzip corpus_mecab.txt.zip


Archive:  corpus_mecab.txt.zip
  inflating: corpus_mecab.txt        


In [3]:
!mkdir embedding


# GloVe install 
https://nlp.stanford.edu/projects/glove/


git clone https://github.com/stanfordnlp/glove  
cd glove && make

## vocab_count  

BUILDING VOCABULARY  
Processed 233118295 tokens.  
Counted 1170689 unique words.  
Truncating vocabulary at min count 5.  
Using vocabulary of size 358042.

In [4]:
#./glove/build/vocab_count -min-count 5 -verbose 2 < corpus_mecab.txt > ./embedding/glove.vocab

BUILDING VOCABULARY
Processed 0 tokens.100000 tokens.200000 tokens.300000 tokens.400000 tokens.500000 tokens.600000 tokens.700000 tokens.800000 tokens.900000 tokens.1000000 tokens.1100000 tokens.1200000 tokens.1300000 tokens.1400000 tokens.1500000 tokens.1600000 tokens.1700000 tokens.1800000 tokens.1900000 tokens.2000000 tokens.2100000 tokens.2200000 tokens.2300000 tokens.2400000 tokens.2500000 tokens.2600000 tokens.2700000 tokens.2800000 tokens.2900000 tokens.3000000 tokens.3100000 tokens.3200000 tokens.3300000 tokens.3400000 tokens.3500000 tokens.3600000 tokens.3700000 tokens.3800000 tokens.3900000 tokens.4000000 tokens.4100000 tokens.4200000 tokens.4300000 tokens.4400000 tokens.4500000 tokens.4600000 tokens.4700000 tokens.4800000 tokens.4900000 tokens.5000000 tokens.5100000 tokens.5200000 tokens.5300000 tokens.5400000 tokens.5500000 tokens.5600000 tokens.5700000 tokens.5800000 tokens.5900000 tokens.6000000 tokens.6100000 tokens.6200000 tokens.6300000 tokens.6400000 tokens.6500000 to

## cooccur  
embedding/glove.vocab -verbose 2 -window-size 15 < corpus_mecab.txt > ./embedding/glove.cooc  
tcmalloc: large alloc 1521139712 bytes == 0x55635e310000 @  
 0x7f8d483b01e7 0x55635ca33462 0x55635ca3212c 0x7f8d479f0bf7 0x55635ca3234a  
COUNTING COOCCURRENCES  
window size: 15  
context: symmetric  
max product: 32676761  
overflow length: 95070890  
Reading vocab from file "./embedding/glove.vocab"...loaded  
358042 words.  
Building lookup table...table contains 302707139 elements.  
tcmalloc: large alloc 2421661696 bytes == 0x5563b9fb6000 @  
 0x7f8d483b2001 0x55635ca33601 0x55635ca3212c 0x7f8d479f0bf7 0x55635ca3234a  
Processing token: 58400000tcmalloc: large alloc 1521139712  
bytes == 0x55644a530000 @  0x7f8d483b01e7 0x7f8d47a116e0 0x55635ca339cd 0x55635ca3212c 0x7f8d479f0bf7 0x55635ca3234a  
Processed 233118295 tokens.  
Writing cooccurrences to disk..........6 files in total.  
Merging cooccurrence files: processed 345407357 lines.  

In [4]:
#./glove/build/cooccur -memory 10.0 -vocab-file ./embedding/glove.vocab -verbose 2 -window-size 15 < corpus_mecab.txt > ./embedding/glove.cooc

tcmalloc: large alloc 1521139712 bytes == 0x558577ada000 @  0x7f9358b641e7 0x558575494492 0x55857549312c 0x7f93581a4bf7 0x55857549334a
COUNTING COOCCURRENCES
window size: 15
context: symmetric
max product: 32676761
overflow length: 95070890
Reading vocab from file "./embedding/glove.vocab"...loaded 358042 words.
Building lookup table...table contains 302707139 elements.
tcmalloc: large alloc 2421661696 bytes == 0x5585d3780000 @  0x7f9358b66001 0x558575494631 0x55857549312c 0x7f93581a4bf7 0x55857549334a
Processing token: 010000020000030000040000050000060000070000080000090000010000001100000120000013000001400000150000016000001700000180000019000002000000210000022000002300000240000025000002600000270000028000002900000300000031000003200000330000034000003500000360000037000003800000390000040000004100000420000043000004400000450000046000004700000480000049000005000000510000052000005300000540000055000005600000570000058000005900000600000061000006200000630000064000006500000660000067000006800000690000

## shuffle
Using random seed 1629275665  
tcmalloc: large alloc 10200547328 bytes == 0x55cbfe19c000 @  0x7fe597a581e7 0x55cbfb9c0d56 0x55cbfb9bfec5 0x7fe597436bf7 0x55cbfb9c004a  
SHUFFLING COOCCURRENCES  
array size: 637534208  
Shuffling by chunks: processed 0 lines.

In [ ]:
#./glove/build/shuffle -memory 10.0 -verbose 2 < ./embedding/glove.cooc > ./embedding/glove.shuf

## Training 
TRAINING MODEL  
Read 345407357 lines.  
Initializing parameters...Using random seed 1629276826  
done.  
vector size: 100  
vocab size: 358042  
x_max: 10.000000  
alpha: 0.750000  
08/18/21 - 09:05.54AM, iter: 001, cost: 0.091448

In [ ]:
#./glove/build/glove -save-file ./embedding/glove.vecs -threads 4 -input-file ./embedding/glove.shuf -x-max 10 -iter 15 -vector-size 100 -binary 2 -vocab-file ./embedding/glove.vocab -verbose 2      

In [6]:
!cp -rf ./embedding '/content/drive/MyDrive/Colab Notebooks/glove'

# 유사도 확인


In [7]:
vecs_fname = './embedding/glove.vecs.txt'
words, vecs = [], []
with open(vecs_fname, 'r', encoding='utf-8') as f:
    for line in f:
        splited_line = line.strip().split(" ")
        words.append(splited_line[0])
        vec = [float(el) for el in splited_line[1:]]
        vecs.append(vec)

In [9]:
from sklearn.preprocessing import normalize
unit_vecs = normalize(vecs, norm='l2', axis=1)

In [10]:
dictionary = {}
for word, vec in zip(words, unit_vecs):
    dictionary[word] = vec

In [21]:
def most_similar_by_vector(query_vec, topn=10):
    query_vec_norm = np.linalg.norm(query_vec)
    if query_vec_norm != 0:
        query_unit_vec = query_vec / query_vec_norm
    else:
        query_unit_vec = query_vec
    scores = np.dot(unit_vecs, query_unit_vec)
    topn_candidates = sorted(zip(words, scores), key=lambda x: x[1], reverse=True)[1:topn+1]
    return topn_candidates

In [22]:
def is_in_vocabulary(word):
  return word in dictionary.keys()

In [23]:
import numpy as np
def get_word_vector(word):  
  if is_in_vocabulary(word):
    vector = dictionary[word]
  else:
    vector = np.zeros(100)
  return vector

In [24]:
def most_similar(query, topn=10):
    query_vec = get_word_vector(query)
    return most_similar_by_vector(query_vec, topn)

In [25]:
most_similar('희망', topn=5)

[('행복', 0.7583373330401103),
 ('사랑', 0.6629804940802159),
 ('꿈', 0.6567677229550366),
 ('미래', 0.6518393400229785),
 ('우리', 0.6518285039960823)]

In [26]:
most_similar('부모', topn=5)

[('자녀', 0.7967711025645954),
 ('가족', 0.7463995232451788),
 ('어머니', 0.7339380229737453),
 ('아버지', 0.7264714754307731),
 ('자식', 0.7211784656053305)]